In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df_list_1_3 = pd.read_csv('results/1-3лист.tsv',  sep='\t')
df_1_3 = pd.read_csv('results/main_table.tsv',  sep='\t')
df_4_bgpd_gbm = pd.read_csv('results/transfer_bgpd_gbm.tsv',  sep='\t')
df_4_gbm_bgpd = pd.read_csv('results/transfer_gbm_bgpd.tsv',  sep='\t')

In [2]:
df_1_3.head()

,PatientID,Preprocessing step,Dataset,"NN-Architecture (nn-Unet, UNETR)","Label (WT, ET, TC, GTV)",Dice,Hausdorff,Surface Dice 1mm,Surface Dice 3mm,Surface Dice 5mm,Epoch,Volume_estimated,Volume_intersection,Volume_true
0,TCGA-02-0011,1_reg,gbm,nn_unet,WT,0.290,39.000,0.264,NaN,NaN,300,25.748,22.114,126.783
1,TCGA-02-0011,1_reg,gbm,nn_unet,ET,0.138,39.000,0.218,NaN,NaN,300,2.983,22.114,33.488
2,TCGA-02-0011,1_reg,gbm,nn_unet,TC,0.030,34.300,0.133,NaN,NaN,300,9.525,22.114,81.619
3,TCGA-02-0033,1_reg,gbm,nn_unet,WT,0.242,39.000,0.261,NaN,NaN,300,19.781,14.996,104.407
4,TCGA-02-0033,1_reg,gbm,nn_unet,ET,0.265,26.094,0.330,NaN,NaN,300,6.324,14.996,31.703


In [9]:
## Table 4: 

reindex_list = ['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan','6_hist','5_ss_shared']
rename_list = ['1. Inter modality', '2. Resampling', '3. Atlas', '4. Resampling to spacing', '4.a Bias', '4.b Denoising', '4.c Histogram', '4.d Skull']


In [14]:
#Table 4 DICE WT
import numpy as np
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
multiplier = 100


def get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(0)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series

    #std
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(0)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series

    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier)


1. Inter modality           44.0 (28.0)
2. Resampling               85.0 (11.0)
3. Atlas                    85.0 (11.0)
4. Resampling to spacing    85.0 (12.0)
4.a Bias                    82.0 (13.0)
4.b Denoising               84.0 (12.0)
4.c Histogram               83.0 (16.0)
4.d Skull                   87.0 (11.0)
Name: Mean (STD), dtype: object

In [11]:
# Appending Table  Hausdorff WT
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'

get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1)


1. Inter modality           31.0 (21.0)
2. Resampling               15.0 (21.0)
3. Atlas                    10.0 (11.0)
4. Resampling to spacing    13.0 (20.0)
4.a Bias                    14.0 (22.0)
4.b Denoising               11.0 (16.0)
4.c Histogram               13.0 (18.0)
4.d Skull                   10.0 (14.0)
Name: Mean (STD), dtype: object

In [12]:
# Appending Table  Surface Dice WT
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'

get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=100)

1. Inter modality           27.0 (15.0)
2. Resampling               65.0 (20.0)
3. Atlas                    63.0 (20.0)
4. Resampling to spacing    65.0 (20.0)
4.a Bias                    60.0 (22.0)
4.b Denoising               64.0 (21.0)
4.c Histogram               63.0 (22.0)
4.d Skull                   69.0 (20.0)
Name: Mean (STD), dtype: object

In [7]:
## How to check where to put rows on the statistical significance:

In [22]:
import numpy as np
preproc_1 = "4a_resamp"
preproc_2 = "1_reg"

data_1 = df_1_3[df_1_3.Dataset=='gbm'][df_1_3['NN-Architecture (nn-Unet, UNETR)']== 'nn_unet'][df_1_3.Epoch==300][df_1_3['Label (WT, ET, TC, GTV)']=='WT'][df_1_3['Preprocessing step']==preproc_1]['Dice']
data_2 = df_1_3[df_1_3.Dataset=='gbm'][df_1_3['NN-Architecture (nn-Unet, UNETR)']== 'nn_unet'][df_1_3.Epoch==300][df_1_3['Label (WT, ET, TC, GTV)']=='WT'][df_1_3['Preprocessing step']==preproc_2]['Dice']

print((np.round(data_1.mean(),3)*100).round(0), (np.round(data_1.std(),3)*100).round(0), (np.round(data_2.mean(),3)*100).round(0), (np.round(data_2.std(),3)*100).round(0),)

85.0 12.0 44.0 28.0


In [15]:
!pip install researchpy

In [51]:
import scipy
from statsmodels.stats.weightstats import ttost_ind
from researchpy import ttest

metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300


preproc_1 = "4a_resamp"
for key in reindex_list:
    preproc_2 = key
    
    if key != "4a_resamp":

        data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
        data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]

        
        ttost = ttost_ind(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
        ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

        # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
        if ttest_ < 0.05:
            print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
              (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
              ' vs ',\
              (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
              (np.round(data_2.std(),3)*100).round(0).astype(int),')')
            print( "Difference is significant, p-value ", np.round(ttest_,3))

1_reg 85 ( 12 )  vs  44 ( 28 )
Difference is significant, p-value  0.0
4b_n4 85 ( 12 )  vs  82 ( 13 )
Difference is significant, p-value  0.0
4d_susan 85 ( 12 )  vs  84 ( 12 )
Difference is significant, p-value  0.025
5_ss_shared 85 ( 12 )  vs  87 ( 11 )
Difference is significant, p-value  0.019
